In [1]:
using Unitful
using UnitfulUS
using Statistics

BoundsError: BoundsError: attempt to access 299-codeunit String at index [1:301]

In [4]:
# Register jupyter notebook as a source of units 
# for new units we define
Unitful.register(Main);
@unit HP "HP" Horsepower 0.7456999*u"kW" false
@unit cc "cc" CubicCentimeter 1u"cm^3" false
@derived_dimension VolumeFlow  Unitful.𝐋^3 / Unitful.𝐓
@derived_dimension SpringConstant Unitful.𝐌*Unitful.𝐋/Unitful.𝐓^2 / Unitful.𝐋

## Sprung corner weight
+ average suspension/wheel is ~45kg for undriven wheel and ~50kg for driven weel
+ If you know that info more precisely, just subtract that from corner weight instead of using this

In [5]:
sprung_corner_weight(corner_weight::Unitful.Mass, driven::Bool) = driven ? corner_weight - 50u"kg" : corner_weight - 45u"kg";

In [6]:
sprung_corner_weight(365u"kg", true)

315 kg

# Target Spring Rate
Info largely from: https://youtu.be/MITt8zpQnFw

### Motion Ratio
Ratio of wheel movement to shock movement. For example, a motion ratio of 1.1:1 means that if the spring moves 1cm the wheel moves 1.1cm. Often different per axel, should calculate separately.
### Target Frequency $\omega$
+ <1 for mild road cars
+ 1.0-1.5 for sporty street cars
+ 1.5-2.5 for low downforce circuit
+ 2.0-3.5 for med downforce circuit
+ 3.5-4.0 for high downforce circuit

In [7]:
function target_spring_rate(sprung_corner_mass, target_ω, motion_ratio)
    no_mr = sprung_corner_mass * 4π^2 * target_ω^2 |> u"N/mm";
    return no_mr * motion_ratio^2;
end

target_spring_rate (generic function with 1 method)

In [8]:
target_spring_rate(338u"kg", 2u"Hz", 1.1) |> u"lbf/inch"

368.7814767875209 lbf inch^-1

# Cutting Springs

In [9]:
function cut_spring_rate(old_rate::SpringConstant, old_length::Unitful.Length, new_length::Unitful.Length)
    return old_rate * old_length / new_length;
end

cut_spring_rate (generic function with 1 method)

In [12]:
cut_spring_rate(368u"lbf/inch", 12u"inch", 11u"inch")

401.45454545454544 lbf inch^-1